In [5]:
!nvidia-smi

Thu May 30 13:22:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        Off | 00000000:18:00.0 Off |                  Off |
| 30%   24C    P8               9W / 450W |     13MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import os
import warnings

warnings.filterwarnings("ignore")
if os.getcwd()[-8:]=='examples':
    os.chdir('..')
    
os.environ["CUDA_VISIBLE_DEVICES"] = "2" # Possible GPUS

In [3]:
notebook = "240125_CIFAR10_WRN16WS_DPSGD-Rebuttal-DG3-3"

In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import importlib
import time

from opacus import PrivacyEngine
from opacus.validators import ModuleValidator

import src
import src.trainer as tr
from src.utils import *

### Settings

In [6]:
#### FOR DP
MAX_GRAD_NORM = 1
EPSILON = 2.0
DELTA = 1e-5

#### FOR TRAINING
LR = 0.5
EPOCHS = 30
BATCH_SIZE = 4096
MAX_PHYSICAL_BATCH_SIZE = 350
PUBLIC_BATCH_SIZE = 100

### FOR MODELING
MODEL_NAME = "WRN16-4_WS"
# MODEL_NAME = "DPNASNet_CIFAR"
DATA = "CIFAR10"
NORMALIZE = {'mean':[0.4930, 0.4875, 0.4548],
             'std':[0.2475, 0.2445, 0.2637]}
N_CLASSES = 10
N_AUGMULT = 16
MAX_PHYSICAL_BATCH_SIZE = int(MAX_PHYSICAL_BATCH_SIZE)


#### FOR SAVE
# tm = time.localtime()
# TIME = time.strftime('%m-%d-%I-%M', tm)

PATH = "./saved/"
if notebook is not "":
    NAME = notebook
else:
    NAME = f"{DATA}_{MODEL_NAME}_LR{LR}_EPS{EPSILON}_C{MAX_GRAD_NORM}_EPOCH{EPOCHS}_BS{BATCH_SIZE}_Augmult{N_AUGMULT}_"

SAVE_PATH = PATH + NAME

### Data loader

In [7]:
public_indices = torch.load('./data/cifar-10-edm/sampled_index.pt')
val_info = [int(i) for i in range(50000) if i in public_indices]

In [8]:
data = src.Datasets(data_name=DATA, train_transform = transforms.ToTensor(),val_info=val_info)

train_loader, public_loader, test_loader = data.get_loader(batch_size=BATCH_SIZE, batch_size_val=PUBLIC_BATCH_SIZE,
                                                           drop_last_train=False, num_workers=16)

Files already downloaded and verified
Files already downloaded and verified
Data Loaded (w/ Validation Set)!
Train Data Length : 48000
Val Data Length : 2000
Test Data Length : 10000


### Model & Optimizer

In [9]:
#### Load model
model = src.utils.load_model(model_name=MODEL_NAME, n_classes=N_CLASSES).cuda() # Load model
model = ModuleValidator.fix(model)
set_groupnorm_num_groups(model, num_groups=16)

ModuleValidator.validate(model, strict=False)

pytorch_total_params = sum(p.numel() for p in model.parameters())

print("model params: {:.4f}M".format(pytorch_total_params/1000000))

#### Define optimizer
optimizer = torch.optim.SGD(model.parameters(),lr=LR, momentum=0)

WRN16-4_WS is loaded.
model params: 2.7489M


### Load PrivacyEngine from Opacus

In [10]:
privacy_engine = PrivacyEngine()

model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    epochs=EPOCHS,
    target_epsilon=EPSILON,
    target_delta=DELTA,
    max_grad_norm=MAX_GRAD_NORM,
)

optimizer.target_epsilon = EPSILON
optimizer.target_delta = DELTA

print(f"Using sigma={optimizer.noise_multiplier} and C={MAX_GRAD_NORM}")
# 2.0664

Using sigma=3.57421875 and C=1


### load pretrained model

In [11]:
rmodel = src.RobModel(model, n_classes=N_CLASSES, normalize=NORMALIZE).cuda()

In [12]:
rmodel.load_dict('../DPTrainer/saved/240127_CIFAR10_WRN16WS_WarmUp-FromDiffusion-DG-Weight3-lr01/best.pth')

Model loaded.
Record Info:
OrderedDict([('Epoch', 90), ('Iter', 782), ('CALoss', 0.003593623172491789), ('CALoss_1', 0.15832000970840454), ('Clean(Tr)', 99.956), ('Clean(Val)', 81.26), ('lr', 0.0029559615522887295), ('s/it', 0.07671212659846548)])


### Start Training

In [13]:
importlib.reload(tr)

trainer = tr.DpTrainer(NAME,rmodel)
trainer.max_physical_batch_size = MAX_PHYSICAL_BATCH_SIZE
trainer.record_rob(train_loader, test_loader)

In [14]:
trainer.fit(train_loader=train_loader, max_epoch=EPOCHS, start_epoch=0,
            optimizer=optimizer,public_loader=public_loader,extender=None,
            scheduler=None, scheduler_type="Epoch",
            minimizer=None, is_ema=True, augmult=N_AUGMULT,
            save_path=SAVE_PATH, save_best={"Clean(Val)":"HB"},
            save_type=None, save_overwrite=True, record_type="Epoch")

[240125_CIFAR10_WRN16WS_DPSGD-Rebuttal-DG3-3]
Training Information.
-Epochs: 30
-Optimizer: SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.5
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)
-Scheduler: None
-Save Path: ./saved/240125_CIFAR10_WRN16WS_DPSGD-Rebuttal-DG3-3
-Save Type: None
-Record Type: Epoch
-Device: cuda:0
----------------------------------------------------------
Epoch   CALoss   Acc(Tr)   Clean(Val)   lr       Sec/it   
1       0.6512   80.0477   82.2400      0.5000   3.8133   
----------------------------------------------------------
2       0.6123   81.2206   82.7000      0.5000   3.8062   
----------------------------------------------------------
3       0.6040   81.7080   83.0600      0.5000   3.7863   
----------------------------------------------------------
4       0.5963   82.1685   83.3200      0.5000   3.8039   
----------------------------------------------------------
5       0

### Evaluation

In [15]:
rmodel.load_dict(PATH+NAME+'/last.pth')
rmodel.eval_accuracy(test_loader)

Model loaded.
Record Info:
OrderedDict([('Epoch', 30), ('CALoss', 0.5117606905690075), ('Acc(Tr)', 85.51459727170041), ('Clean(Val)', 85.54), ('lr', 0.5)])


85.53

In [16]:
rmodel.load_dict(PATH+NAME+'/best.pth')
rmodel.eval_accuracy(test_loader)

Model loaded.
Record Info:
OrderedDict([('Epoch', 30), ('CALoss', 0.5117606905690075), ('Acc(Tr)', 85.51459727170041), ('Clean(Val)', 85.54), ('lr', 0.5)])


85.54